In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import json

In [3]:
tmdb_movies = pd.read_csv('data/tmdb/tmdb_5000_movies.csv')
tmdb_credits = pd.read_csv('data/tmdb/tmdb_5000_credits.csv')

In [4]:
tmdb_movies.head(3)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466


In [5]:
tmdb_credits.head(3)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."


In [6]:
print(tmdb_movies.shape)
print(tmdb_credits.shape)

(4803, 20)
(4803, 4)


# FIND SIMILARTY FOR OVERVIEWS

In [7]:
tmdb_credits.columns = ['id','title','cast','crew']
tmdb_credits.head(3)

,id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."


In [8]:
tmdb_all_data = tmdb_movies.merge(tmdb_credits, on='id')
tmdb_all_data.head(3)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title_x,vote_average,vote_count,title_y,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."


In [9]:
tmdb_all_data.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title_x', 'vote_average',
       'vote_count', 'title_y', 'cast', 'crew'],
      dtype='object')

In [10]:
tmdb_all_data['overview'] = tmdb_all_data['overview'].fillna('')

In [11]:
tfidf = TfidfVectorizer(stop_words='english')

In [12]:
transformed_overviews = tfidf.fit_transform(tmdb_all_data['overview'])
transformed_overviews.shape

(4803, 20978)

### We use linear kernel to find similarity between overviews because we need dot product here, and it gives us the cosine similarity score.

In [13]:
cosine_similarities = linear_kernel(transformed_overviews, transformed_overviews)
cosine_similarities.shape

(4803, 4803)

In [14]:
def get_movie_recommendation_with_similarities(title, similarity, top=10):
    idx = tmdb_all_data[tmdb_all_data['title_x'] == title].index[0]
    sim_scores = list(enumerate(similarity[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:top+1]
    movie_indices = [i[0] for i in sim_scores]
    return pd.DataFrame(tmdb_all_data['title_x'].iloc[movie_indices])
    

### Let's get a random movie from the dataset and find the top 10 similar movies.

In [15]:
random_movie = tmdb_all_data.sample(1)['title_x'].values[0]
print('Movie: ', random_movie)
get_movie_recommendation_with_similarities(random_movie, cosine_similarities, top=10)

Movie:  Cellular


,title_x
3106,High Heels and Low Lifes
4030,The Other End of the Line
4162,Buried
1513,Transporter 2
3850,The Betrayed
4744,Happy Christmas
2892,Darling Companion
4098,Ghost Dog: The Way of the Samurai
1460,The Horseman on the Roof
4189,Run Lola Run


### We will use the other features to find the similarity.

#### We will use count vectorizer to convert other features to one vector. Let's see how it works.

In [16]:
vectorizer = CountVectorizer()
sampleData = ['This is the first document.','This document is the second document.','And this is the third one.','Is this the first document?',]
X = vectorizer.fit_transform(sampleData)
print(vectorizer.get_feature_names())
print(X.toarray())

['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']
[[0 1 1 1 0 0 1 0 1]
 [0 2 0 1 0 1 1 0 1]
 [1 0 0 1 1 0 1 1 1]
 [0 1 1 1 0 0 1 0 1]]


/Applications/anaconda3/envs/390/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [17]:
vectorizer = CountVectorizer()

### Now we know how to convert the features to vectors. Let's concat all features to one data and convert it to vector.

### Get cast, crew, keywords and genre names from the dataset.

In [18]:
new_data = tmdb_all_data.copy()
new_data['cast'] = new_data['cast'].apply(lambda x: [i['name'] for i in json.loads(x)] if isinstance(x, str) else [])
new_data['crew'] = new_data['crew'].apply(lambda x: [i['name'] for i in json.loads(x)] if isinstance(x, str) else [])
new_data['genres'] = new_data['genres'].apply(lambda x: [i['name'] for i in json.loads(x)] if isinstance(x, str) else [])
new_data['keywords'] = new_data['keywords'].apply(lambda x: [i['name'] for i in json.loads(x)] if isinstance(x, str) else [])

new_data.head(3)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title_x,vote_average,vote_count,title_y,cast,crew
0,237000000,"[Action, Adventure, Fantasy, Science Fiction]",http://www.avatarmovie.com/,19995,"[culture clash, future, space war, space colon...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Avatar,"[Sam Worthington, Zoe Saldana, Sigourney Weave...","[Stephen E. Rivkin, Rick Carter, Christopher B..."
1,300000000,"[Adventure, Fantasy, Action]",http://disney.go.com/disneypictures/pirates/,285,"[ocean, drug abuse, exotic island, east india ...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Pirates of the Caribbean: At World's End,"[Johnny Depp, Orlando Bloom, Keira Knightley, ...","[Dariusz Wolski, Gore Verbinski, Jerry Bruckhe..."
2,245000000,"[Action, Adventure, Crime]",http://www.sonypictures.com/movies/spectre/,206647,"[spy, based on novel, secret agent, sequel, mi...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Spectre,"[Daniel Craig, Christoph Waltz, Léa Seydoux, R...","[Thomas Newman, Sam Mendes, Anna Pinnock, John..."


### Convert related features to strings for count vectorizer.

In [19]:
new_data['genres'] = new_data['genres'].apply(lambda x: ' '.join(x))
new_data['cast'] = new_data['cast'].apply(lambda x: ' '.join(x))
new_data['crew'] = new_data['crew'].apply(lambda x: ' '.join(x))
new_data['keywords'] = new_data['keywords'].apply(lambda x: ' '.join(x))
new_data.head(3)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title_x,vote_average,vote_count,title_y,cast,crew
0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Avatar,Sam Worthington Zoe Saldana Sigourney Weaver S...,Stephen E. Rivkin Rick Carter Christopher Boye...
1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Pirates of the Caribbean: At World's End,Johnny Depp Orlando Bloom Keira Knightley Stel...,Dariusz Wolski Gore Verbinski Jerry Bruckheime...
2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6 bri...,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Spectre,Daniel Craig Christoph Waltz Léa Seydoux Ralph...,Thomas Newman Sam Mendes Anna Pinnock John Log...


### Add all features to one data.

In [20]:
new_data['concated_data'] = new_data['genres'] + ' ' + new_data['cast'] + ' ' + new_data['crew'] + ' ' + new_data['keywords']
new_data.head(3)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,spoken_languages,status,tagline,title_x,vote_average,vote_count,title_y,cast,crew,concated_data
0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Avatar,Sam Worthington Zoe Saldana Sigourney Weaver S...,Stephen E. Rivkin Rick Carter Christopher Boye...,Action Adventure Fantasy Science Fiction Sam W...
1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Pirates of the Caribbean: At World's End,Johnny Depp Orlando Bloom Keira Knightley Stel...,Dariusz Wolski Gore Verbinski Jerry Bruckheime...,Adventure Fantasy Action Johnny Depp Orlando B...
2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6 bri...,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Spectre,Daniel Craig Christoph Waltz Léa Seydoux Ralph...,Thomas Newman Sam Mendes Anna Pinnock John Log...,Action Adventure Crime Daniel Craig Christoph ...


In [21]:
count_matrix = vectorizer.fit_transform(new_data['concated_data'])
count_matrix.shape

(4803, 62236)

In [22]:
cosine_similarity_for_concated_data = cosine_similarity(count_matrix, count_matrix)

### Let's see how it looks like with some random movies.

In [23]:
random_movie = tmdb_all_data.sample(1)['title_x'].values[0]
print('Movie: ', random_movie)
get_movie_recommendation_with_similarities(random_movie, cosine_similarity_for_concated_data, top=10)

Movie:  Welcome to the Rileys


,title_x
3649,"Lovely, Still"
1424,Concussion
2034,Delivery Man
1703,Role Models
928,Moneyball
3858,Stranded
334,Jurassic Park III
1759,The Right Stuff
3506,Don McKay
199,Pirates of the Caribbean: The Curse of the Bla...


In [24]:
random_movie = tmdb_all_data.sample(1)['title_x'].values[0]
print('Movie: ', random_movie)
get_movie_recommendation_with_similarities(random_movie, cosine_similarity_for_concated_data, top=10)

Movie:  October Baby


,title_x
3705,Moms' Night Out
4624,Locker 13
3486,Goddess of Love
3102,Rosewater
733,Up Close & Personal
3623,Made
4770,Mutual Appreciation
2081,Molly
1241,Sweet November
78,The Jungle Book


### Let's see more random movies.

In [25]:
for i in range(10):
    random_movie = tmdb_all_data.sample(1)['title_x'].values[0]
    print('Movie: ', random_movie)
    print(get_movie_recommendation_with_similarities(random_movie, cosine_similarity_for_concated_data, top=10))
    print('-------------------------')

Movie:  Transsiberian
                           title_x
3635                 The Machinist
4234                     Session 9
4358          Next Stop Wonderland
3397                          Ca$h
40                          Cars 2
4066                         [REC]
1635       The Replacement Killers
4794  Sanctuary: Quite a Conundrum
2572                 Boogie Nights
1328                      The Town
-------------------------
Movie:  The Hitchhiker's Guide to the Galaxy
                            title_x
752             My Favorite Martian
292                          Eragon
1279                   Stormbreaker
4332                 Silent Running
266                        I, Robot
901   Robin Hood: Prince of Thieves
1129         The Mothman Prophecies
2592            Highlander: Endgame
718                   Deep Blue Sea
1825      Jimmy Neutron: Boy Genius
-------------------------
Movie:  xXx: State of the Union
                                       title_x
1133                 

## Collaborative Filtering

Unlike content-based filtering, collaborative filtering does not require item metadata like genre, overview, cast, crew, etc. It works by learning the user's preferences and recommending items that are similar to the user's preferences. The underlying assumption is that if a user liked a particular item, he or she will also like an item that is similar to it.

We're going to use the  [Surprise](https://surpriselib.com/) library to implement collaborative filtering. Surprise is a Python scikit building and analyzing recommender systems that deal with explicit rating data. It also has implementations of various other algorithms for recommendation systems.

In [26]:
from surprise import Reader, Dataset, SVD

import warnings; warnings.simplefilter('ignore')
from surprise.model_selection import cross_validate
from sklearn.model_selection import train_test_split


In [27]:
reader = Reader()

In [28]:
ratings = pd.read_csv('./data/movies-dataset/ratings_small.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [29]:
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

In [30]:
svd = SVD()
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9017  0.8935  0.8957  0.8965  0.8964  0.8968  0.0027  
MAE (testset)     0.6944  0.6896  0.6908  0.6901  0.6871  0.6904  0.0024  
Fit time          0.42    0.41    0.39    0.40    0.39    0.40    0.01    
Test time         0.09    0.05    0.09    0.05    0.09    0.07    0.02    


{'test_rmse': array([0.90170757, 0.89354638, 0.89574514, 0.89649653, 0.89640369]),
 'test_mae': array([0.69443385, 0.68963429, 0.69079643, 0.69007116, 0.68711377]),
 'fit_time': (0.4214611053466797,
  0.4131019115447998,
  0.3910861015319824,
  0.3980541229248047,
  0.390625),
 'test_time': (0.09007906913757324,
  0.04989314079284668,
  0.0859372615814209,
  0.04597282409667969,
  0.08675193786621094)}

In [51]:
trainset = data.build_full_trainset()
svd.fit(trainset)

In [42]:
svd.predict(1, 302, 3)

Prediction(uid=1, iid=302, r_ui=3, est=2.7293618417270564, details={'was_impossible': False})

In [114]:
def hybrid(userId, title):
    similars = get_movie_recommendation_with_similarities(title, cosine_similarity_for_concated_data, top=26)
    similar_indices = similars.index.tolist()

    movies = tmdb_all_data.iloc[similar_indices][['original_title', 'vote_count', 'vote_average', 'id']]
    movies['est'] = movies['id'].apply(lambda x: svd.predict(userId, x).est)
    movies = movies.sort_values('est', ascending=False)
    return movies.head(10)

In [116]:
get_movie_recommendation_with_similarities('Avatar', cosine_similarity_for_concated_data, top=20)

,title_x
28,Jurassic World
25,Titanic
1133,15 Minutes
3,The Dark Knight Rises
363,A.I. Artificial Intelligence
65,The Dark Knight
185,War of the Worlds
508,The Lost World: Jurassic Park
47,Star Trek Into Darkness
16,The Avengers


In [118]:
hybrid(5, 'Avatar')

,original_title,vote_count,vote_average,id,est
1133,15 Minutes,191,5.7,2749,4.098967
25,Titanic,7562,7.5,597,4.079780
169,Captain America: The First Avenger,7047,6.6,1771,4.013861
28,Jurassic World,8662,6.5,135397,3.770010
280,Public Enemies,1344,6.5,11322,3.770010
38,The Amazing Spider-Man 2,4179,6.5,102382,3.770010
412,The Core,516,5.4,9341,3.770010
714,Collateral,1451,7.0,1538,3.770010
218,Jason Bourne,2341,5.9,324668,3.770010
94,Guardians of the Galaxy,9742,7.9,118340,3.770010
